### Part1 : 
### Prepare input data for diagnosis portion 

In [1]:
import pandas as pd
import spacy
import random
import re

In [2]:
nlp = spacy.load('es_core_news_md')

In [4]:
def get_sentence_vectors(text, nlp):
    
    # get tokens for each word in a sentence
    embedding = nlp(text).vector.tolist()
    
    return embedding



Import and merge data

In [23]:
illness_df = pd.read_csv('diaganosis symtom/dia_t.csv', sep='\t')
symptom_df = pd.read_csv('diaganosis symtom/sym_t.csv', sep='\t')
links_df = pd.read_csv('diaganosis symtom/diffsydiw.csv')  # Este normalmente sigue siendo CSV

In [25]:
illness_df.head()

,did,diagnose
0,1,Aneurisma aórtico abdominal (agrandamiento del...
1,2,Hinchazón abdominal
2,3,Trauma abdominal
3,4,Abrasiones (raspaduras)
4,5,Tos inducida por inhibidor de la ECA; efecto s...


In [26]:
symptom_df.head()

,syd,symptom
0,1,Dolor abdominal superior
1,2,Dolor abdominal inferior
2,3,Absceso; acumulación de pus
3,4,Abuso de alcohol
4,5,Ansiedad; nerviosismo


In [28]:
source_data = (links_df
               .merge(illness_df, on="did")
               .merge(symptom_df, on="syd"))
source_data

,syd,did,wei,diagnose,symptom
0,1.0,163.0,2.0,Colecistitis; inflamación de la vesícula biliar,Dolor abdominal superior
1,1.0,164.0,2.0,Coledocolitiasis; cálculo en el conducto biliar,Dolor abdominal superior
2,1.0,165.0,1.0,Colelitiasis; cálculos biliares,Dolor abdominal superior
3,1.0,187.0,2.0,Estreñimiento,Dolor abdominal superior
4,1.0,306.0,2.0,Úlcera gástrica; úlcera estomacal,Dolor abdominal superior
...,...,...,...,...,...
5544,106.0,827.0,0.0,Carcinoma escamoso vulvar; cáncer de piel en g...,Secreción vaginal
5545,186.0,966.0,2.0,Pérdida de cabello; alopecia,Pérdida de cabello (calvicie)
5546,186.0,1415.0,0.0,Hipoparatiroidismo,Pérdida de cabello (calvicie)
5547,186.0,1415.0,0.0,Hipoparatiroidismo,Pérdida de cabello (calvicie)


In [29]:
source_data

,syd,did,wei,diagnose,symptom
0,1.0,163.0,2.0,Colecistitis; inflamación de la vesícula biliar,Dolor abdominal superior
1,1.0,164.0,2.0,Coledocolitiasis; cálculo en el conducto biliar,Dolor abdominal superior
2,1.0,165.0,1.0,Colelitiasis; cálculos biliares,Dolor abdominal superior
3,1.0,187.0,2.0,Estreñimiento,Dolor abdominal superior
4,1.0,306.0,2.0,Úlcera gástrica; úlcera estomacal,Dolor abdominal superior
...,...,...,...,...,...
5544,106.0,827.0,0.0,Carcinoma escamoso vulvar; cáncer de piel en g...,Secreción vaginal
5545,186.0,966.0,2.0,Pérdida de cabello; alopecia,Pérdida de cabello (calvicie)
5546,186.0,1415.0,0.0,Hipoparatiroidismo,Pérdida de cabello (calvicie)
5547,186.0,1415.0,0.0,Hipoparatiroidismo,Pérdida de cabello (calvicie)


In [32]:
print(source_data.columns)

Index(['illness_id', 'symptom_id', 'illness', 'symptom'], dtype='object')


In [33]:
# Remove missing data and select columns we needed
# Filtra filas sin datos faltantes en illness o symptom
source_data = source_data.loc[
    ~(source_data['symptom'].isna()) & ~(source_data['illness'].isna()),
    ['illness_id', 'symptom_id', 'illness', 'symptom']
]

In [34]:
source_data

,illness_id,symptom_id,illness,symptom
0,163.0,1.0,Colecistitis; inflamación de la vesícula biliar,Dolor abdominal superior
1,164.0,1.0,Coledocolitiasis; cálculo en el conducto biliar,Dolor abdominal superior
2,165.0,1.0,Colelitiasis; cálculos biliares,Dolor abdominal superior
3,187.0,1.0,Estreñimiento,Dolor abdominal superior
4,306.0,1.0,Úlcera gástrica; úlcera estomacal,Dolor abdominal superior
...,...,...,...,...
5544,827.0,106.0,Carcinoma escamoso vulvar; cáncer de piel en g...,Secreción vaginal
5545,966.0,186.0,Pérdida de cabello; alopecia,Pérdida de cabello (calvicie)
5546,1415.0,186.0,Hipoparatiroidismo,Pérdida de cabello (calvicie)
5547,1415.0,186.0,Hipoparatiroidismo,Pérdida de cabello (calvicie)


In [35]:
# tidy up some new messy characters
source_data['illness'] = source_data['illness'].str.replace('\x0b', ' ')
source_data['symptom'] = source_data['symptom'].str.replace('\x0b', ' ')

In [36]:
source_data

,illness_id,symptom_id,illness,symptom
0,163.0,1.0,Colecistitis; inflamación de la vesícula biliar,Dolor abdominal superior
1,164.0,1.0,Coledocolitiasis; cálculo en el conducto biliar,Dolor abdominal superior
2,165.0,1.0,Colelitiasis; cálculos biliares,Dolor abdominal superior
3,187.0,1.0,Estreñimiento,Dolor abdominal superior
4,306.0,1.0,Úlcera gástrica; úlcera estomacal,Dolor abdominal superior
...,...,...,...,...
5544,827.0,106.0,Carcinoma escamoso vulvar; cáncer de piel en g...,Secreción vaginal
5545,966.0,186.0,Pérdida de cabello; alopecia,Pérdida de cabello (calvicie)
5546,1415.0,186.0,Hipoparatiroidismo,Pérdida de cabello (calvicie)
5547,1415.0,186.0,Hipoparatiroidismo,Pérdida de cabello (calvicie)


Use our function to convert each symptom to a vector representation we can then save our symptom data down to the data folder

In [37]:
symptom_df = symptom_df.loc[~symptom_df['symptom'].isna()]

symptom_df['embedding'] = symptom_df.apply(lambda row: get_sentence_vectors(row['symptom'], nlp), axis=1)
symptom_df.columns = ['symptom_id', 'symptom', 'symptom_vector']

In [38]:
symptom_df

,symptom_id,symptom,symptom_vector
0,1,Dolor abdominal superior,"[0.8097400069236755, 0.03385166451334953, 0.02..."
1,2,Dolor abdominal inferior,"[0.7883666157722473, 0.15227466821670532, 0.17..."
2,3,Absceso; acumulación de pus,"[0.29806798696517944, 1.2345305681228638, 2.88..."
3,4,Abuso de alcohol,"[0.16878998279571533, 0.43853434920310974, 2.5..."
4,5,Ansiedad; nerviosismo,"[-0.1033099964261055, 2.0011000633239746, -0.9..."
...,...,...,...
300,302,Infertilidad (femenina),"[-2.62823486328125, -1.2763324975967407, 0.206..."
301,303,Infertilidad (femenina),"[-2.62823486328125, -1.2763324975967407, 0.206..."
302,304,Presión arterial alta,"[2.503166437149048, -1.7192333936691284, -0.10..."
303,305,Infertilidad (femenina),"[-2.62823486328125, -1.2763324975967407, 0.206..."


In [39]:
# remove messy charecters

symptom_df['symptom'] = symptom_df['symptom'].str.replace('\x0b', ' ')

In [40]:
source_data.to_pickle('input_data/source_data.pkl')
symptom_df.to_pickle('input_data/symptoms.pkl')


### Create a vector of symtpoms for each illness described in the data

Here we will loop through each illness described in the dataset
and flag (with a 1 or 0) each of the symptoms of that illness. 
We will do then compare a list of flagged symptoms described by 
the user to this using a cosine similarity function to diagnose
a potential illness.

In [42]:
# list of illness 
illnesses = list(source_data['illness'].drop_duplicates())

# list we will use store our illness vectors 
symptom_vectors = []

for illness in illnesses:
    
    illnesses_symptoms = list(source_data.loc[source_data['illness'] == illness, 'symptom'].drop_duplicates())
    
    symptom_df['related_to_illness'] = 0
    symptom_df.loc[symptom_df['symptom'].isin(illnesses_symptoms), "related_to_illness"] = 1
    
    symptom_vectors.append(list(symptom_df["related_to_illness"]))
    
diagnosis_data = pd.DataFrame({"illness":illnesses,
                                "illness_vector":symptom_vectors})
diagnosis_data.to_pickle('input_data/diagnosis_data.pkl')


### Part 2:
### Generate training samples with flagged entities

In this part we will attempt to generate some training examples for the NLU model to understand how our users will describe symtoms to the chat bot. To do this we simply loop through our symptoms, sometimes combining them before appending them to different beginnings/endings.

We also make sure to tag any symptom describtions as a symptom entity, which will be understood by Rasa's NLU model. You can see this is accomplished by inserting '(symptom)' beside any mention of symptoms.

The output of this is inserted into nlu.md within the main project.

In [46]:
import random
import re

# Frases de inicio en español
inicios_descripcion = [
    "Tengo",
    "Estoy sufriendo de",
    "Me siento muy mal con",
    "Mis síntomas son",
    "Desde hace unos días tengo",
    "Mi esposo sufre de",
    "Mi esposa sufre de",
    "Mi hijo sufre de",
    "Mi hija sufre de",
    "Mi niño/a sufre de",
    "No me siento bien, tengo"
]

numero_de_sintomas = [1, 2, 3, 4]

for cantidad in numero_de_sintomas:
    for ex in range(1, 101):
        inicio = random.choice(inicios_descripcion)
        
        # Selecciona síntomas aleatorios
        sintoma_1 = symptom_df['symptom'].sample(1).iloc[0].lower()
        sintoma_2 = symptom_df['symptom'].sample(1).iloc[0].lower()
        sintoma_3 = symptom_df['symptom'].sample(1).iloc[0].lower()
        sintoma_4 = symptom_df['symptom'].sample(1).iloc[0].lower()
        
        sintomas = [sintoma_1, sintoma_2, sintoma_3, sintoma_4]
        sintomas_entidad = []
        
        # Limpia paréntesis y marca como entidad
        for sintoma in sintomas:
            sintoma = re.sub(r"\([^)]+\)", "", sintoma).strip()
            sintoma = f"[{sintoma}](symptom)"
            sintomas_entidad.append(sintoma)
        
        sintoma_1 = sintomas_entidad[0]
        sintoma_2 = sintomas_entidad[1]
        sintoma_3 = sintomas_entidad[2]
        sintoma_4 = sintomas_entidad[3]
        
        # Construye la frase según la cantidad de síntomas
        if cantidad == 1:
            frase = f"- {inicio} {sintoma_1}"
        elif cantidad == 2:
            frase = f"- {inicio} {sintoma_1} y {sintoma_2}"
        elif cantidad == 3:
            frase = f"- {inicio} {sintoma_1}, {sintoma_2} y {sintoma_3}"
        elif cantidad == 4:
            frase = f"- {inicio} {sintoma_1}, {sintoma_2}, {sintoma_3} y {sintoma_4}"
        
        print(frase)

- Estoy sufriendo de [dolor de cuello](symptom)
- No me siento bien, tengo [dolor escrotal](symptom)
- No me siento bien, tengo [falta de aire](symptom)
- No me siento bien, tengo [ronquidos](symptom)
- Mis síntomas son [picazón en el brazo](symptom)
- Me siento muy mal con [dolor de cabeza y debilidad](symptom)
- Mi niño/a sufre de [parálisis facial](symptom)
- Tengo [hinchazón de labios](symptom)
- Me siento muy mal con [secreción del oído](symptom)
- Mi esposo sufre de [hinchazón del pie](symptom)
- Mi hija sufre de [dolor de cabeza después de un trauma](symptom)
- Mi hija sufre de [hinchazón abdominal](symptom)
- Mi niño/a sufre de [pérdida de equilibrio](symptom)
- Desde hace unos días tengo [cianosis](symptom)
- Mi hija sufre de [escalofríos](symptom)
- No me siento bien, tengo [dolor o sensibilidad en el pecho](symptom)
- Estoy sufriendo de [retención urinaria](symptom)
- Desde hace unos días tengo [enrojecimiento ocular](symptom)
- Desde hace unos días tengo [dolor de oído](sym